# 策略对比汇总（Classic vs ML）

这个 notebook 用来把：
- 经典策略（MACD/RSI/Bollinger）的回测统计
- 以及 `run_lstm2_pipeline.py` 输出的 ML 回测统计（JSON）

汇总成一张对比表，方便写中期报告。


In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd

from fyp_trading.data import fetch_prices
from fyp_trading.backtest import equity_and_stats_from_positions
from fyp_trading.strategies import (
    MacdConfig, macd_crossover_signal,
    RsiConfig, rsi_mean_reversion_signal,
    BollingerConfig, bollinger_mean_reversion_signal,
)

ROOT = Path(".").resolve()
REPORTS_DIR = ROOT / "outputs" / "reports"

TICKER = "2800.HK"
PERIOD = "3y"
INTERVAL = "1d"
COST_BP = 2.0
ALLOW_SHORT = False

# 1) classic strategies

df = fetch_prices(TICKER, PERIOD, INTERVAL)
close = df["Close"].copy()
next_ret = np.log(close.shift(-1) / close)
simple_ret = (np.exp(next_ret) - 1.0).dropna()

results = []

# MACD
sig = macd_crossover_signal(close, MacdConfig(allow_short=ALLOW_SHORT))
_, macd_stats = equity_and_stats_from_positions(simple_ret, sig.loc[simple_ret.index], COST_BP)
results.append({"strategy": "MACD", **macd_stats})

# RSI
sig = rsi_mean_reversion_signal(close, RsiConfig(allow_short=ALLOW_SHORT))
_, rsi_stats = equity_and_stats_from_positions(simple_ret, sig.loc[simple_ret.index], COST_BP)
results.append({"strategy": "RSI", **rsi_stats})

# Bollinger
sig = bollinger_mean_reversion_signal(close, BollingerConfig(allow_short=ALLOW_SHORT))
_, bb_stats = equity_and_stats_from_positions(simple_ret, sig.loc[simple_ret.index], COST_BP)
results.append({"strategy": "Bollinger", **bb_stats})

# 2) ML (read latest json)
ml_jsons = sorted(REPORTS_DIR.glob("lstm2_backtest_stats_*.json"))
if ml_jsons:
    latest = ml_jsons[-1]
    with latest.open("r", encoding="utf-8") as f:
        ml_stats = json.load(f)
    results.append({"strategy": f"ML_LSTM2 ({latest.name})", **ml_stats})
else:
    print("No ML backtest stats found. Run: python run_lstm2_pipeline.py")

summary = pd.DataFrame(results)
cols = [
    "strategy",
    "total_return",
    "annualized_return",
    "annualized_volatility",
    "sharpe_ratio",
    "max_drawdown",
    "coverage",
    "transaction_cost_bp",
]
summary = summary[[c for c in cols if c in summary.columns]]
summary
